# Watermass transformation in the CDW density range



Watermass transformation rates are defined as:

\begin{eqnarray}
    \Omega &=& \frac{\partial}{\partial \sigma} \int \int \int \frac{D \sigma'}{D t} dV, \\
    \Omega(\sigma, t)_{i,j} &=& A_c \frac{1}{\Delta \sigma} \sum^{N_z} \bigg (  \frac{D \sigma}{D t}  h_c \Delta z_f \delta(\sigma - \sigma') \bigg ),
\end{eqnarray}

$$ \frac{D \sigma}{Dt} = \frac{\partial \sigma}{\partial \theta} \dot{\theta} + \frac{\partial \sigma}{\partial S}\dot{S} $$

$ \alpha = -\frac{1}{\rho} \frac{\partial \sigma}{\partial \theta}$ and $\beta = \frac{1}{\rho} \frac{\partial \sigma}{\partial S_A}$

$$\dot{\theta} = \frac{D \theta}{Dt} = G^\theta_{hdiff} + G^\theta_{vdiff} + G^\theta_{surf} + G^\theta_{SW}$$

$$\dot{S} = \frac{D S}{Dt} = G^S_{hdiff} + G^S_{vdiff} + G^S_{surf}$$


Since we have the theta and salinity advective terms available as online outputs at monthly frequency, we can use that to directly obtain the 

$$\dot{\theta} = \frac{\delta \theta}{\Delta t} + \frac{\delta (u\theta \Delta z \Delta y)}{\Delta x \Delta y \Delta z} + \frac{\delta (v\theta \Delta z \Delta x)}{\Delta x \Delta y \Delta z}$$ 

Where $\delta$ is the difference operator.



In [1]:
%config Completer.use_jedi = False

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import cmocean as cm
import xarray as xr
import numpy as np
import IPython.display
import cosima_cookbook as cc
import pandas as pd
import gsw

In [3]:
session = cc.database.create_session()

In [4]:
expList = cc.querying.get_experiments(session)

In [99]:
expList.loc[expList["experiment"].str.contains("01deg_jra55v140_iaf_cycle4")]

,experiment,ncfiles
94,01deg_jra55v140_iaf_cycle4,131990
156,01deg_jra55v140_iaf_cycle4_jra55v150_extension,13422
184,01deg_jra55v140_iaf_cycle4_rerun_from_1983,172
185,01deg_jra55v140_iaf_cycle4_rerun_from_1986,86
188,01deg_jra55v140_iaf_cycle4_rerun_from_1980,2582


In [6]:
experiment = "01deg_jra55v140_iaf_cycle4"

In [101]:
fileList = cc.querying.get_ncfiles(session, experiment)

In [37]:
varList = cc.querying.get_variables(session, experiment=experiment, frequency="1 monthly")

In [91]:
varList["frequency"].unique()

array([None, '1 daily', '1 monthly', 'static'], dtype=object)

In [41]:
varList.loc[varList["long_name"].str.lower().str.contains("cell")]#.loc[163, "ncfile"]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
2,HTE,T cell width on East side,m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
3,HTN,T cell width on North side,m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
29,buoyfreq2_wt,Squared buoyancy frequency at T-cell bottom,1/s^2,1 monthly,output991/ocean/ocean-3d-buoyfreq2_wt-1-monthl...,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
39,dxt,T cell width through middle,m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
40,dxu,U cell width through middle,m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
41,dyt,T cell height through middle,m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
42,dyu,U cell height through middle,m,1 monthly,output991/ice/OUTPUT/iceh.2018-12.nc,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
43,dzt,t-cell thickness,m,1 monthly,output991/ocean/ocean-3d-dzt-1-monthly-mean-ym...,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
67,grid_xt_ocean,tcell longitude,degrees_E,1 monthly,output991/ocean/ocean-3d-ty_trans_rho-1-monthl...,None,1464,1958-01-01 00:00:00,2019-01-01 00:00:00
68,grid_xu_ocean,ucell longitude,degrees_E,1 monthly,output991/ocean/ocean-3d-tx_trans_rho-1-monthl...,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00


In [42]:
st_ocean

<xarray.DataArray 'st_ocean' (st_ocean: 75)>
array([5.412807e-01, 1.680735e+00, 2.939953e+00, 4.331522e+00, 5.869350e+00,
       7.568810e+00, 9.446885e+00, 1.152234e+01, 1.381593e+01, 1.635055e+01,
       1.915154e+01, 2.224687e+01, 2.566746e+01, 2.944746e+01, 3.362460e+01,
       3.824057e+01, 4.334140e+01, 4.897796e+01, 5.520640e+01, 6.208874e+01,
       6.969342e+01, 7.809601e+01, 8.737988e+01, 9.763699e+01, 1.089687e+02,
       1.214869e+02, 1.353144e+02, 1.505868e+02, 1.674530e+02, 1.860765e+02,
       2.066365e+02, 2.293296e+02, 2.543701e+02, 2.819920e+02, 3.124492e+02,
       3.460166e+02, 3.829906e+02, 4.236883e+02, 4.684475e+02, 5.176241e+02,
       5.715899e+02, 6.307275e+02, 6.954248e+02, 7.660668e+02, 8.430255e+02,
       9.266482e+02, 1.017244e+03, 1.115068e+03, 1.220309e+03, 1.333076e+03,
       1.453384e+03, 1.581154e+03, 1.716205e+03, 1.858264e+03, 2.006975e+03,
       2.161913e+03, 2.322601e+03, 2.488533e+03, 2.659189e+03, 2.834054e+03,
       3.012631e+03, 3.194453e+03, 3.379089e+03, 3.566145e+03, 3.755274e+03,
       3.946166e+03, 4.138551e+03, 4.332197e+03, 4.526903e+03, 4.722497e+03,
       4.918835e+03, 5.115794e+03, 5.313270e+03, 5.511177e+03, 5.709443e+03])
Coordinates:
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
Attributes:
    long_name:       tcell zstar depth
    units:           meters
    cartesian_axis:  Z
    positive:        down
    edges:           st_edges_ocean

In [ ]:
dzt = cc.querying.getvar(experiment, "dzt", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [18]:
average_DT = cc.querying.getvar(experiment, "average_DT", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

0.3.0


In [7]:
sw_ocean = cc.querying.getvar(experiment, "sw_ocean", session, frequency="1 monthly")

0.3.0


In [8]:
pot_rho_0 = cc.querying.getvar(experiment, "pot_rho_0", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [9]:
pot_rho_0 = pot_rho_0.sel(yt_ocean = slice(-90, -60))

In [10]:
temp = cc.querying.getvar(experiment, "temp", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

temp = temp.sel(yt_ocean = slice(-90, -60))

temp_xflux_adv = cc.querying.getvar(experiment, "temp_xflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

temp_xflux_adv = temp_xflux_adv.sel(yt_ocean = slice(-90, -60))

temp_yflux_adv = cc.querying.getvar(experiment, "temp_yflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

temp_yflux_adv = temp_yflux_adv.sel(yu_ocean = slice(-90, -60))

salt = cc.querying.getvar(experiment, "salt", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt = salt.sel(yt_ocean = slice(-90, -60))

salt_xflux_adv = cc.querying.getvar(experiment, "salt_xflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt_xflux_adv = salt_xflux_adv.sel(yt_ocean = slice(-90, -60))

salt_yflux_adv = cc.querying.getvar(experiment, "salt_yflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")


In [11]:
salt_yflux_adv = salt_yflux_adv.sel(yu_ocean = slice(-90, -60))

In [12]:
ds = xr.Dataset(coords = {"yt_ocean":(["yt_ocean"], temp.yt_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "yu_ocean":(["yu_ocean"], temp_yflux_adv.yu_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "xt_ocean":(["xt_ocean"], temp.xt_ocean.values, {"axis":"X", "c_grid_axis_shift":None}),
                          "xu_ocean":(["xu_ocean"], temp_xflux_adv.xu_ocean.values, {"axis":"X", "c_grid_axis_shift": -0.5})
                         })

In [13]:
delta_lat_t2t = np.diff(ds["yt_ocean"])
delta_lat_t2t = np.append(delta_lat_t2t, delta_lat_t2t[-1])

delta_lat_u2u = np.diff(ds["yu_ocean"].values)
delta_lat_u2u = np.append(delta_lat_u2u, delta_lat_u2u[-1])

In [14]:
Re = 6370e3 # Radius of the earth in meters

In [15]:
delta_y_t2t = np.abs(Re * np.deg2rad(delta_lat_t2t)) # delta(lat) converted to distance in meters 
delta_y_u2u = np.abs(Re * np.deg2rad(delta_lat_u2u)) # delta(lat) converted to distance in meters 

In [16]:
delta_lon_t2t = np.diff(ds["xt_ocean"])
delta_lon_t2t = np.append(delta_lon_t2t, delta_lon_t2t[-1])

In [17]:
delta_lon_u2u = np.diff(ds["xu_ocean"])
delta_lon_u2u = np.append(delta_lon_u2u, delta_lon_u2u[-1])

In [18]:
delta_x_t2t = np.zeros((ds["yt_ocean"].shape[-1], ds["xu_ocean"].shape[-1]))
for i in range(len(ds["yt_ocean"].values)):
    delta_x_t2t[i] = np.abs(Re * np.cos( np.deg2rad(ds["yt_ocean"].values[i]) ) * np.deg2rad(delta_lon_t2t))

In [19]:
delta_x_u2u = np.zeros((ds["yt_ocean"].shape[-1], ds["xt_ocean"].shape[-1]))
for i in range(len(ds["yt_ocean"].values)):
    delta_x_u2u[i] = np.abs(Re * np.cos( np.deg2rad(ds["yt_ocean"].values[i]) ) * np.deg2rad(delta_lon_u2u))

In [20]:
st_ocean = temp.st_ocean
delta_s_t2t = np.diff(st_ocean)
delta_s_t2t = np.append(delta_s_t2t, delta_s_t2t[-1])

In [21]:
delta_s_w2w = np.diff(sw_ocean)
delta_s_w2w = np.append(delta_s_w2w, delta_s_w2w[-1])

In [22]:
time_vars = temp.time

In [23]:
ds = xr.Dataset(coords = {"yt_ocean":(["yt_ocean"], temp.yt_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "yu_ocean":(["yu_ocean"], temp_yflux_adv.yu_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "xt_ocean":(["xt_ocean"], temp.xt_ocean.values, {"axis":"X", "c_grid_axis_shift":None}),
                          "xu_ocean":(["xu_ocean"], temp_xflux_adv.xu_ocean.values, {"axis":"X", "c_grid_axis_shift": -0.5}),
                          "st_ocean":(["st_ocean"], temp.st_ocean.values, {"axis":"Z", "c_grid_axis_shift": None}),
                          "sw_ocean":(["sw_ocean"], sw_ocean.values, {"axis":"Z", "c_grid_axis_shift": -0.5}),

                          "delta_s_t2t":(["sw_ocean"], delta_s_t2t, {"axis":"Z", "c_grid_axis_shift":-0.5}),
                          "delta_s_w2w":(["st_ocean"], delta_s_w2w, {"axis":"Z", "c_grid_axis_shift":None}),
                          "delta_y_t2t":(["yu_ocean"], delta_y_t2t, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "delta_y_u2u":(["yt_ocean"], delta_y_u2u, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "delta_x_t2t":(["yt_ocean", "xu_ocean"], delta_x_t2t),
                          "delta_x_u2u":(["yt_ocean", "xt_ocean"], delta_x_u2u),
                         })

In [24]:
from xgcm import Grid

In [25]:
xgrid = Grid(ds, 
            metrics={("X",):["delta_x_t2t", "delta_x_u2u"],
                     ("Y",):["delta_y_t2t", "delta_y_u2u"],
                     ("Z"):["delta_s_t2t", "delta_s_w2w"]
            })

In [26]:
volume_cell_t = ds["delta_x_u2u"] * ds["delta_y_u2u"] * ds["delta_s_w2w"]

In [27]:
years_to_step_by = np.arange(1990, 1991, 1)

In [28]:
years_to_step_by

array([1990])

In [29]:
tarea = ds["delta_x_u2u"] * ds["delta_y_u2u"]

In [30]:
sigma_intervals = np.linspace(27.67, 27.9, 20)
sigma_delta = sigma_intervals[1] - sigma_intervals[0]


In [31]:
temp = temp.sel(time = slice("1990", "2019")).chunk({"time":12})
salt = salt.sel(time = slice("1990", "2019")).chunk({"time":12})
temp_xflux_adv = temp_xflux_adv.sel(time = slice("1990", "2019")).chunk({"time":12})
temp_yflux_adv = temp_yflux_adv.sel(time = slice("1990", "2019")).chunk({"time":12})
salt_xflux_adv = salt_xflux_adv.sel(time = slice("1990", "2019")).chunk({"time":12})
salt_yflux_adv = salt_yflux_adv.sel(time = slice("1990", "2019")).chunk({"time":12})
pot_rho_0 = pot_rho_0.sel(time = slice("1990", "2019")).chunk({"time":12})

In [51]:

for y in years_to_step_by:
    print(y)

    # create an Omega for each year (~2.5GB). Omega is the WMT
    Omega = np.full_like(np.zeros((12, len(sigma_intervals)-1, len(temp.yt_ocean.values), len(temp.xt_ocean.values) )), fill_value=0.)
    
    Cp, rho0 = 4e3, 1035.0

    Dtemp_Dt = temp.sel(time=str(y)).differentiate("time", datetime_unit="s") + xgrid.diff(temp_xflux_adv.sel(time=str(y)), "X") / (volume_cell_t*Cp*rho0) + \
               xgrid.diff(temp_yflux_adv.sel(time=str(y)), "Y") / (volume_cell_t*Cp*rho0)

    Dsalt_Dt = salt.sel(time=str(y)).chunk((12, 19, 135, 180)).differentiate("time", datetime_unit="s") + xgrid.diff(salt_xflux_adv.sel(time=str(y)), "X") / (rho0*volume_cell_t) + \
               xgrid.diff(salt_yflux_adv.sel(time=str(y)), "Y") / (rho0*volume_cell_t)

    DsaltAbs_Dt = gsw.SA_from_SP(Dsalt_Dt, st_ocean, Dsalt_Dt.xt_ocean, Dsalt_Dt.yt_ocean)

    absSalt = gsw.SA_from_SP(salt, st_ocean, salt.xt_ocean, salt.yt_ocean)

    alpha = gsw.alpha(absSalt, temp, st_ocean)

    beta = gsw.beta(absSalt, temp, st_ocean)

    Dsigma_Dt_temp = -rho0 * alpha * Dtemp_Dt

    Dsigma_Dt_salt = rho0 * beta * DsaltAbs_Dt

    Dsigma_Dt = Dsigma_Dt_temp + Dsigma_Dt_salt



1990


In [ ]:
348 * 19 * 400 * 3600

In [ ]:
(len(temp.time.values), len(sigma_intervals)-1, len(temp.yt_ocean.values), len(temp.xt_ocean.values) )

In [ ]:

for si in range(len(sigma_intervals)-1):
    sigma_seld = pot_rho_0.sel(time =slice(ts,te))
    Omega[si] =  (tarea * (Dsigma_Dt.\
                    where((sigma_seld < (sigma_intervals[si] + sigma_delta*0.5)) & 
                          (sigma_seld > (sigma_intervals[si] - sigma_delta*0.5)) ) * 
                                                ds["delta_s_w2w"]).sel(st_ocean=slice(0,1000)).sum("st_ocean") /\
                                  sigma_delta).fillna(0.0) #fillna to avoid propagating NaNs; 0 to 700m because potential density approximates a neutral surface within ~500m of it's reference surface. Also, we are looking at near-surface processes here.

# Experimental and trial work below

In [7]:
x = np.arange(0, 51, 5)
delta_x = np.diff(x)
y = np.linspace(0, 10, x.shape[-1])
delta_y = np.diff(y)

u = np.random.randn(delta_x.shape[-1])
theta = np.random.randn(delta_x.shape[-1]) + 10

In [10]:
np.diff(u*theta*delta_y) / (delta_x*delta_y)[:-1]

array([-4.48262572, -1.4165097 ,  4.2525832 , -2.62312913, -3.42340953,
        2.75148562, -2.05581626, -2.02625185,  6.77570832])

In [11]:
np.diff(u*theta)/delta_x[:-1]

array([-4.48262572, -1.4165097 ,  4.2525832 , -2.62312913, -3.42340953,
        2.75148562, -2.05581626, -2.02625185,  6.77570832])